In [1]:
#dependencies
import pandas as pd
import pgeocode
from config import gkey
import requests
import json
import pymongo
import numpy as np

In [2]:
#connect to mongo db and create "locations_mdb" database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.locations_mdb

In [4]:
#build the zip code reference table
zips_df=pd.read_csv('sf_zips.csv')
zips_df["Latitude"] = ""
zips_df["Longitude"] = ""
tool = pgeocode.Nominatim('US')

for index, row in zips_df.iterrows():
    zipcode = row['ZIPCode']
    tool_table=tool.query_postal_code(zipcode)
    lat=tool_table["latitude"]
    long=tool_table["longitude"]
    zips_df.loc[index, 'Latitude']=lat
    zips_df.loc[index, 'Longitude']=long
    

<h2>Kathryn's Data</h2>
<h4>"Hotels" collection:<h4>
<p>The hotels collection is based on a query of every hotel located within 50,000 meters from the latitude and longitude corresponding to each zip code in the original dataset</p>

In [4]:
###############
#KATHRYN'S DATA
###############
#HOTELS collection: build from google api data
#load into mongoDB

api_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in zips_df.iterrows():
    target_coordinates = f"{zips_df.loc[index, 'Latitude']},{zips_df.loc[index, 'Longitude']}" 
    target_radius = 50000
    target_type = "lodging"

# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey}

# run a request using our params dictionary
    response = requests.get(api_url, params=params).json()
    
    
    db.hotels.insert_one(
        {
            'zip_code': row['ZIPCode'],
            'type': row['Type'],
            'county': row['County'],
            'population': row['Population'],
            'area_codes': row['Area Code(s)'],
            'latitude': row['Latitude'],
            'longitude': row['Longitude'],
            'hotels': response["results"]
        }
    )

<h2>Heesung's Data</h2>
<h4>"Boba" collection:<h4>
<p>The Boba collection is based on (Heesung's description here)</p>

In [5]:
###############
#HEESUNG'S DATA
###############
#BOBA collection: build from Heesung's json file 
#load into mongoDB

collection_boba = db['boba']
with open ("../Heesung_resources/boba_pika.json") as f:
    file_data = json.load(f)
    for record in file_data:
        collection_boba.insert_one(record)

<h2>Caitlin's Data</h2>
<h4>"Parks" collection:<h4>
<p>The Parks collection is based on (Caitlin's description here)</p>

In [6]:
###############
#CAITLIN'S DATA
###############
#PARKS collection: build from Caitlin's json file (built from CSV file earlier)
#load into mongoDB

collection_parks = db['parks']
parks_df = pd.read_csv ('../Cait_resources/Resources/SF_Park_Scores.csv')
parks_dict=parks_df.to_dict('index')

dumps=json.dumps(parks_dict)
load=json.loads(dumps)
collection_parks.insert_one(load)

<h2>William's Data</h2>
<h4>"Restaurants" collection:<h4>
<p>The Parks collection is based on (Caitlin's description here)</p>

In [7]:
###############
#WILLIAM'S DATA
###############
#RESTAURANTS collection: build from William's CSV file (json was formatted strangely)
#load into mongoDB

collection_restaurants = db['restaurants']
restaurants_df = pd.read_csv ('../Resources/raw.csv')
restaurants_dict=restaurants_df.to_dict('index')

dumps=json.dumps(restaurants_dict)
load=json.loads(dumps)
collection_restaurants.insert_one(load)

In [8]:
collection_parks = db['parks']
parks_df = pd.read_csv ('../Cait_resources/Resources/SF_Park_Scores.csv')
parks_dict=parks_df.to_dict('index')

dumps=json.dumps(parks_dict)
load=json.loads(dumps)
collection_parks.insert_one(load)

<h1>Searching the Database for Nested Values</h1>

<p>loop through the data to find desired value pairs using the method below.</p>

In [165]:

#for location in db.locations.find({"hotels.rating":4.1}):
#    for hotel in location["hotels"]:
#        if hotel["rating"] == 4.1:
#            print(json.dumps(hotel, sort_keys=True, indent=2))
            